# Sports Betting Arbitrage Program
<u> TLDR </u>: skip to "Main" section.  

The purpose of this project is to create an abritrage sports betting bot that identifies arbitrage opportunities determines the how much to bet on each of the outcmoes to gaurantee profit.

---
The program currrently: 
- identifies arbitrage opportunities in real time using a sports betting API
- displays results of arbitrage opportunites as well as how much to bet for each team along with their expected profit    

This notebook is split into two sections: 
- <u>Functions</u>: the definition and explanations of functions.
- <u>Main</u>: the section where you can actually **run the program to search for arbitrage opportunities**.  

You can customise (in the "Main" section):
- the amount of money you wish to visualise the arbitrage for 
- whether or not you want the arbitrage results of all matches (yes and no) or just the ones that have arbitrage opportunites (yes only)  
- whether or not you want the results of only matches that have not become a live game yet (pre-game period)

Potential improvements: 
- incoporate the second bets from Betfair
---


## Functions ##

In [1]:
import pandas as pd 
import requests as rq
import json
import time
import datetime

    choose_dataset(name)
<u>Description</u>: determines whether we should gt the dataset from a specified text file or from a live API pull request.  

<u>Parameters</u>:
- name: the name of the option being choosed. 

<u>Returns</u>: 
- df_api: the dataframe containing the sports market data. 
- pull_from: a string indicating whether we pulled from a live API or from a text file. 

In [2]:
def choose_dataset(database_name: str) -> [pd.DataFrame, str]: 
    """
    Chooses which dataset to use: either a text file or a live API call.
    """
    if database_name == 'The Odds API':
        # #australianmtg API key 
        # url = "https://api.the-odds-api.com/v4/sports/upcoming/odds/?regions=au&markets=h2h&apiKey=64ddb7c6d547c7407c3405b99eb7cd89"

        #vincentgiang.aus API key 
        url = "https://api.the-odds-api.com/v4/sports/upcoming/odds/?regions=au&markets=h2h&apiKey=c4d0da2a3053259ee7b9f829345f6f76"
        
        name = "The Odds API"
        file = rq.get(url)
        file_content = file.json()

        #Convert the API data into a dataframe
        df_api = pd.DataFrame(file_content)

        #To print out at the end
        pull_from = f"Data pulled from API: '{name}'"
        
        return df_api, pull_from
        
    elif database_name == 'exampleAPIcall' or database_name == 'exampleAPIcall2' or database_name == 'nobookieAPIcall':
        folder = "example api calls (text files)"
        file_name = f"{folder}/{database_name}.txt"

        with open(file_name, 'r') as file:
            # Convert the text file (string) into recognised dictionary format
            file_content = json.load(file)

        #Convert the API data into a dataframe
        df_api = pd.DataFrame(file_content)

        #To print out at the end 
        pull_from = f"Data pulled from text file: '{file_name}'"
        
        return df_api, pull_from

    parse_bookmaker_data(bookmaker, df_bookmaker)
<u>Description</u>: parses the data retrieved from the API, adding it to a dataframe for use.  

<u>Parameters</u>:
- bookmaker: a dictionary of the current bookmaker of interest and their properties.
- df_bookmaker: a dataframe containing all of the bookmakers and their properties for this specific game.  

<u>Returns</u>: 
- None

In [3]:
def parse_bookmaker_data(bookmaker: dict, df_bookmaker: pd.DataFrame) -> None:
    """
    Parses the API information into a dataframe.
    """
    #Extract details of this specific bookmaker for this specific game 
    bookmaker_name = bookmaker['title']
    prices = [outcome['price'] for outcome in bookmaker['markets'][0]['outcomes']] #[team1, team2, draw]
    team1 = bookmaker["markets"][0]["outcomes"][0]['name']
    team2 = bookmaker["markets"][0]["outcomes"][1]['name']
    market_key = bookmaker["markets"][0]["key"]
    last_updated = f"{bookmaker["markets"][0]["last_update"][11:-1]} UTC on {bookmaker["markets"][0]["last_update"][0:10]}"

    #Add the bookmaker's details to the dataframe for this specific game
    data = {'bookmaker': bookmaker_name, 'prices': prices, 'team1': team1, 'team2': team2, "market_key": market_key, "last_updated": last_updated}
    df_bookmaker.loc[len(df_bookmaker)] = data

    find_best_odds(df_bookmaker)
<u>Description</u>: find the bookmakers offering the best (highest) odds for each team.

<u>Parameters</u>:
- df_bookmaker: a dataframe containing all of the bookmakers and their properties for this specific game.  

<u>Returns</u>: 
- df_best_team1_odds.T: a dataframe containing the bookmaker that offers the best odds for team 1.
- df_best_team2_odds.T: a dataframe containing the bookmaker that offers the best odds for team 2.
- df_best_draw_odds.T: a dataframe containing the bookmaker that offers the best odds for a draw (or sometimes 'None' when the game does not have draw odds).

In [4]:

def find_best_odds(df_bookmaker: pd.DataFrame) -> [pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    """
    Determines the bookmakers with the best odds for each team.
    """
    #Create a new data frame for the prices. 3 columns when bookmaker includes odds for when the result is a 'draw'.
    df_prices = pd.DataFrame(df_bookmaker['prices'].tolist())
    if len(df_prices.columns) == 2:
        df_prices.columns = ['team1', 'team2']
    else:
        df_prices.columns = ['team1', 'team2', 'draw']

    #Find the index of the row that has the highest odds for each team/draw
    idx_best_team1 = df_prices['team1'].idxmax()
    idx_best_team2 = df_prices['team2'].idxmax()

    #Find the bookmaker that has the best odds for team 1, extract that row. 
    data_team1_odds = df_bookmaker.loc[idx_best_team1, ['bookmaker', 'prices', 'team1', 'market_key', 'last_updated']]
    data_team1_odds['prices'] = data_team1_odds['prices'][0]
    best_team1_odds = pd.DataFrame(data_team1_odds)

    #Find the bookmaker that has the best odds for team 2, extract that row. 
    data_team2_odds = df_bookmaker.loc[idx_best_team2, ['bookmaker', 'prices', 'team2', 'market_key', 'last_updated']]
    data_team2_odds['prices'] = data_team2_odds['prices'][1]
    best_team2_odds = pd.DataFrame(data_team2_odds)

    #Find the bookmaker that has the best odds for a draw, extract that row. 
    if len(df_prices.columns) == 3:
        idx_best_draw = df_prices['draw'].idxmax()
        data_draw_odds = df_bookmaker.loc[idx_best_draw, ['bookmaker', 'prices', 'market_key', 'last_updated']]
        data_draw_odds['prices'] = data_draw_odds['prices'][2]
        best_draw_odds = pd.DataFrame(data_draw_odds)
        return best_team1_odds.T, best_team2_odds.T, best_draw_odds.T

    return best_team1_odds.T, best_team2_odds.T, None

    calculate_arbitrage(df_best_team1_odds, df_best_team2_odds)
<u>Description</u>: calculate if an arbitrage exists between the teams with the best odds. 

<u>Parameters</u>:
- df_best_team1_odds: a dataframe containing the bookmaker that offers the best odds for team 1.
- df_best_team2_odds: a dataframe containing the bookmaker that offers the best odds for team 2.
- df_best_draw_odds: a dataframe containing the bookmaker that offers the best odds for a draw.

<u>Returns</u>: 
- implied_probability: the arbitrage percentage (1/team1odds + 1/team2odds)
- team1_probability: the probability of team 1 (1/team1odds)
- team2_probability: the probability of team 2 (1/team2odds)
- draw_probability: the probability of a draw (1/drawodds)

In [5]:
def calculate_arbitrage(df_best_team1_odds: pd.DataFrame, df_best_team2_odds: pd.DataFrame, df_best_draw_odds: pd.DataFrame) -> [float, float, float, float]:
    """"
    Calculates if an arbitrage opportunity exists.
    """
    #This bookmaker has odds for a draw 
    if df_best_draw_odds is not None: 
        team1_probability = 1/df_best_team1_odds['prices'].iloc[0]
        team2_probability = 1/df_best_team2_odds['prices'].iloc[0]
        draw_probability = 1/df_best_draw_odds['prices'].iloc[0]

        #Calculate the arbitrage percentage
        implied_probability = team1_probability + team2_probability + draw_probability
        return implied_probability, team1_probability, team2_probability, draw_probability
    
    #This bookmaker does not have odds for a draw
    else: 
        team1_probability = 1/df_best_team1_odds['prices'].iloc[0]
        team2_probability = 1/df_best_team2_odds['prices'].iloc[0]

        #Calculate the arbitrage percentage
        implied_probability = team1_probability + team2_probability
        return implied_probability, team1_probability, team2_probability, None

    UTC_to_AEDT(input_time)
<u>Description</u>: convert a string in UTC to an ouput in AEDT time.

<u>Parameters</u>:
- input_time: a string with the time needed to be converted in UTC

<u>Returns</u>: 
- output_time: a string with the time in AEDT

In [6]:
def UTC_to_AEDT(input_time: str) -> str:
    """ 
    Given a input time, convert it and return it in AEDT format.
    """
    utc_datetime = datetime.datetime.strptime(input_time, "%Y-%m-%dT%H:%M:%SZ")
    aedt_datetime = utc_datetime + datetime.timedelta(hours=11)
    output_time = aedt_datetime.strftime("%I:%M:%S %p on %d-%m-%Y")
    return output_time

    display_results(detailed, df_results, implied_probability, df_best_team1_odds, df_best_team2_odds, df_best_draw_odds, match_details, game_number, number_of_games, team1_probability, team2_probability, stake, list_arbitrage_opportunites, draw_probability, show_live)
<u>Description</u>: displays the results of whether or not an arbitrage exists and the bets to place to capitalise on it.

<u>Parameters</u>:
- detailed: whether or not we want to display the results of all matches or just ones with arbitrage.
- df_results: a dataframe that the results of an arbitrage opportunity will be appended to. These opportunities will then be sorted later.
- implied_probability: the arbitrage percentage (1/team1odds + 1/team2odds)
- df_best_team1_odds: a dataframe containing the bookmaker that offers the best odds for team 1.
- df_best_team2_odds: a dataframe containing the bookmaker that offers the best odds for team 2.
- df_best_draw_odds: a dataframe containing the bookmaker that offers the best odds for a draw.
- match_details: the details of this specific match such as the sport and which teams are playing.
- game_number: a counter to keep track of the current game number.
- number_of_games: the total number of games in this API call.
- team1_probability: the probability of team 1 (1/team1odds)
- team2_probability: the probability of team 2 (1/team2odds)
- stake: the amount of money intended to be wagered.
- list_arbitrage_opportunities: a list of the current arbitrage opportunities to be printed out when 'detailed' == False.
- draw_probability: the probability of a draw (1/drawodds)
- show_live: a boolean telling us whether or not to display live results.

<u>Returns</u>: 
- None

In [7]:

def display_results(detailed: bool, df_results: pd.DataFrame, implied_probability: float, df_best_team1_odds: pd.DataFrame, df_best_team2_odds: pd.DataFrame, df_best_draw_odds: pd.DataFrame, match_details: str, game_number: int, number_of_games: int, team1_probability: float, team2_probability: float, stake: int, list_arbitrage_opportunites: list, draw_probability: float, show_live: bool) -> None:
    """
    Displays the results of the program depending based on two key parameters: 
    1. detailed: whether or not we want to display the results of every game (T) or only the results of the games with arbitrage opportunities (F)
    2. show_live: whether or not we want to display the results of games regardless of commence time (T) or only the results of games with commence time in the future (games in the pre-game period)

    Code structure:
    - (!!) detailed = T (show the results of every game)
    - (!!) detailed = F (show only arbitrage results) 
        - (!) show_live = T (show all arbitrage games regardless of match commence time) 
        - (!) show_live = F (show all arbitrage games that are in the pre-game period)
        - (!) if it is the last game, also print out the results

    """
    #Convert time from UTC to AEDT for Team 1
    temp_time = datetime.datetime.strptime(df_best_team1_odds['last_updated'].iloc[0], "%H:%M:%S UTC on %Y-%m-%d")
    formatted_aedt_datetime_team1 = UTC_to_AEDT(temp_time.strftime("%Y-%m-%dT%H:%M:%SZ"))

    #Convert time from UTC to AEDT for Team 2
    temp_time = datetime.datetime.strptime(df_best_team2_odds['last_updated'].iloc[0], "%H:%M:%S UTC on %Y-%m-%d")
    formatted_aedt_datetime_team2 = UTC_to_AEDT(temp_time.strftime("%Y-%m-%dT%H:%M:%SZ"))

    #Convert time from UTC to AEDT for draw odds
    if df_best_draw_odds is not None: 
        temp_time = datetime.datetime.strptime(df_best_draw_odds['last_updated'].iloc[0], "%H:%M:%S UTC on %Y-%m-%d")
        formatted_aedt_datetime_draw = UTC_to_AEDT(temp_time.strftime("%Y-%m-%dT%H:%M:%SZ"))
        
    #Get current time 
    current_time_utc = datetime.datetime.now(datetime.timezone.utc)
    formatted_current_time_utc = current_time_utc.strftime("%Y-%m-%dT%H:%M:%SZ")
    formatted_aedt_datetime_current = UTC_to_AEDT(formatted_current_time_utc)
    
    #Get current time ready to compare against match time 
    compare_current_time = datetime.datetime.strptime(formatted_aedt_datetime_current, "%I:%M:%S %p on %d-%m-%Y")
    
    #Get match time
    match_time_start = match_details.find("Match Time:")
    formatted_aedt_datetime_match_time = match_details[match_time_start + len("Match Time:"):].strip()
    
    #Get match time ready to compare against current time
    compare_match_time = datetime.datetime.strptime(formatted_aedt_datetime_match_time, "%I:%M:%S %p on %d-%m-%Y")
    
    #(!!) Show the results of every game, taking into account ignore_draw_odds but regardless of show_live and arbitrage outcome 
    if detailed:
        #There is no arbitrage opportunity
        print(f"---------------------------------------------------------------------\nMatch {game_number}: {match_details}")
        if implied_probability > 1: 
            #There is draw for this game.
            if draw_probability is not None:
                print(f"NO arbitrage opportunity exists ({round(implied_probability, 4)})\n[{df_best_team1_odds['team1'].iloc[0]}: {round(1/df_best_team1_odds['prices'].iloc[0],4)}, {df_best_team2_odds['team2'].iloc[0]}: {round(1/df_best_team2_odds['prices'].iloc[0],4)}, Draw : {round(1/df_best_draw_odds['prices'].iloc[0],4)}]\n")
            #There are no draw odds for this game.
            else: 
                    print(f"NO arbitrage opportunity exists ({round(implied_probability, 4)})\n[{df_best_team1_odds['team1'].iloc[0]} : {round(1/df_best_team1_odds['prices'].iloc[0],4)}, {df_best_team2_odds['team2'].iloc[0]} : {round(1/df_best_team2_odds['prices'].iloc[0],4)}]\n")
        #There is an arbitrage opportunity
        else: 
            if draw_probability is not None: 
                print(f"Arbitrage opportunity exists ({round(implied_probability,4)})\n\nStake: ${stake}\nBet ${round(team1_probability*stake/implied_probability,2)} on *{df_best_team1_odds['team1'].iloc[0]}* on *{df_best_team1_odds['bookmaker'].iloc[0]}* ({df_best_team1_odds['prices'].iloc[0]}) ({df_best_team1_odds['market_key'].iloc[0]}) (Last Updated: {formatted_aedt_datetime_team1})\nBet ${round(team2_probability*stake/implied_probability,2)} on *{df_best_team2_odds['team2'].iloc[0]}* on *{df_best_team2_odds['bookmaker'].iloc[0]}* ({df_best_team2_odds['prices'].iloc[0]}) ({df_best_team2_odds['market_key'].iloc[0]}) (Last Updated: {formatted_aedt_datetime_team2})\nBet ${round(draw_probability*stake/implied_probability,2)} on *Draw* on *{df_best_draw_odds['bookmaker'].iloc[0]}* ({df_best_draw_odds['prices'].iloc[0]}) ({df_best_draw_odds['market_key'].iloc[0]}) (Last Updated: {formatted_aedt_datetime_draw})\nExpected Profit: ${round(stake/implied_probability-stake,2)}\n")
            else:
                print(f"Arbitrage opportunity exists ({round(implied_probability,4)})\n\nStake: ${stake}\nBet ${round(team1_probability*stake/implied_probability,2)} on *{df_best_team1_odds['team1'].iloc[0]}* on *{df_best_team1_odds['bookmaker'].iloc[0]}* ({df_best_team1_odds['prices'].iloc[0]}) ({df_best_team1_odds['market_key'].iloc[0]}) (Last Updated: {formatted_aedt_datetime_team1})\nBet ${round(team2_probability*stake/implied_probability,2)} on *{df_best_team2_odds['team2'].iloc[0]}* on *{df_best_team2_odds['bookmaker'].iloc[0]}* ({df_best_team2_odds['prices'].iloc[0]}) ({df_best_team2_odds['market_key'].iloc[0]}) (Last Updated: {formatted_aedt_datetime_team2})\nExpected Profit: ${round(stake/implied_probability-stake,2)}\n")
            
            list_arbitrage_opportunites.append(f"Match: {game_number}")

    #(!!) Show the results of games with arbitrage, taking into account show_live
    else: 
        
        #(!) Show results of all games (in-game and pre-game)
        if show_live: 
            #There is an arbitrage opportunity. Append arbitrage opportunity to a list to print out later (at the end of the last game)
            if implied_probability < 1:
                #Check if there is an option to draw or not. Below is for when there is draw.
                if draw_probability is not None: 
                    arbitrage_data = f"Match {game_number}: {match_details}\nArbitrage opportunity exists ({round(implied_probability,4)})\n\nStake: ${stake}\nBet ${round(team1_probability*stake/implied_probability,2)} on *{df_best_team1_odds['team1'].iloc[0]}* on *{df_best_team1_odds['bookmaker'].iloc[0]}* ({df_best_team1_odds['prices'].iloc[0]}) ({df_best_team1_odds['market_key'].iloc[0]}) (Last Updated: {formatted_aedt_datetime_team1})\nBet ${round(team2_probability*stake/implied_probability,2)} on *{df_best_team2_odds['team2'].iloc[0]}* on *{df_best_team2_odds['bookmaker'].iloc[0]}* ({df_best_team2_odds['prices'].iloc[0]}) ({df_best_team2_odds['market_key'].iloc[0]}) (Last Updated: {formatted_aedt_datetime_team2})\nBet ${round(draw_probability*stake/implied_probability,2)} on *Draw* on *{df_best_draw_odds['bookmaker'].iloc[0]}* ({df_best_draw_odds['prices'].iloc[0]}) ({df_best_draw_odds['market_key'].iloc[0]}) (Last Updated: {formatted_aedt_datetime_draw})\nExpected Profit: ${round(stake/implied_probability-stake,2)}\n"
                else:
                    arbitrage_data = f"Match {game_number}: {match_details}\nArbitrage opportunity exists ({round(implied_probability,4)})\n\nStake: ${stake}\nBet ${round(team1_probability*stake/implied_probability,2)} on *{df_best_team1_odds['team1'].iloc[0]}* on *{df_best_team1_odds['bookmaker'].iloc[0]}* ({df_best_team1_odds['prices'].iloc[0]}) ({df_best_team1_odds['market_key'].iloc[0]}) (Last Updated: {formatted_aedt_datetime_team1})\nBet ${round(team2_probability*stake/implied_probability,2)} on *{df_best_team2_odds['team2'].iloc[0]}* on *{df_best_team2_odds['bookmaker'].iloc[0]}* ({df_best_team2_odds['prices'].iloc[0]}) ({df_best_team2_odds['market_key'].iloc[0]}) (Last Updated: {formatted_aedt_datetime_team2})\nNo draw odds.\nExpected Profit: ${round(stake/implied_probability-stake,2)}\n"

                #Add the data to a dataframe to order later
                expected_profit = stake/implied_probability-stake
                data = {'expected_profit': expected_profit, 'arbitrage_data': arbitrage_data}
                df_results.loc[len(df_results)] = data

                list_arbitrage_opportunites.append(f'Match: {game_number}')
        
        #(!) Show results of non-live games only (pre-game only)
        else:
            #Check to see if the current game is during the pre-game period.
            if compare_match_time >= compare_current_time:
                if implied_probability < 1:
                    #Check if there is an option to draw or not. Below is for when there is draw.
                    if draw_probability is not None: 
                        arbitrage_data = f"Match {game_number}: {match_details}\nArbitrage opportunity exists ({round(implied_probability,4)})\n\nStake: ${stake}\nBet ${round(team1_probability*stake/implied_probability,2)} on *{df_best_team1_odds['team1'].iloc[0]}* on *{df_best_team1_odds['bookmaker'].iloc[0]}* ({df_best_team1_odds['prices'].iloc[0]}) ({df_best_team1_odds['market_key'].iloc[0]}) (Last Updated: {formatted_aedt_datetime_team1})\nBet ${round(team2_probability*stake/implied_probability,2)} on *{df_best_team2_odds['team2'].iloc[0]}* on *{df_best_team2_odds['bookmaker'].iloc[0]}* ({df_best_team2_odds['prices'].iloc[0]}) ({df_best_team2_odds['market_key'].iloc[0]}) (Last Updated: {formatted_aedt_datetime_team2})\nBet ${round(draw_probability*stake/implied_probability,2)} on *Draw* on *{df_best_draw_odds['bookmaker'].iloc[0]}* ({df_best_draw_odds['prices'].iloc[0]}) ({df_best_draw_odds['market_key'].iloc[0]}) (Last Updated: {formatted_aedt_datetime_draw})\nExpected Profit: ${round(stake/implied_probability-stake,2)}\n"
                    else:
                        arbitrage_data = f"Match {game_number}: {match_details}\nArbitrage opportunity exists ({round(implied_probability,4)})\n\nStake: ${stake}\nBet ${round(team1_probability*stake/implied_probability,2)} on *{df_best_team1_odds['team1'].iloc[0]}* on *{df_best_team1_odds['bookmaker'].iloc[0]}* ({df_best_team1_odds['prices'].iloc[0]}) ({df_best_team1_odds['market_key'].iloc[0]}) (Last Updated: {formatted_aedt_datetime_team1})\nBet ${round(team2_probability*stake/implied_probability,2)} on *{df_best_team2_odds['team2'].iloc[0]}* on *{df_best_team2_odds['bookmaker'].iloc[0]}* ({df_best_team2_odds['prices'].iloc[0]}) ({df_best_team2_odds['market_key'].iloc[0]}) (Last Updated: {formatted_aedt_datetime_team2})\nNo draw odds.\nExpected Profit: ${round(stake/implied_probability-stake,2)}\n"

                    #Add the data to a dataframe to order later
                    expected_profit = stake/implied_probability-stake
                    data = {'expected_profit': expected_profit, 'arbitrage_data': arbitrage_data}
                    df_results.loc[len(df_results)] = data

                    list_arbitrage_opportunites.append(f'Match: {game_number}')              
        
        #(!) For cases when we only want the games with arbitrage, if it is the last game print out the results (which were previously appended to a list)
        if game_number == number_of_games:
            #Order by the highest profit and print results
            df_results = df_results.sort_values(by = 'expected_profit', ascending=False)
            for index, game in df_results.iterrows():
                print(f"---------------------------------------------------------------------\n{game['arbitrage_data']}")
    
    

    increment_tracker()
<u>Description</u>: increment the tracker that tracks the number of API calls this month.

<u>Parameters</u>:
- None

<u>Returns</u>: 
- None

In [8]:
def increment_tracker() -> None:
    """
    Increment the tracker for the number of API calls.
    """
    # Read the current value from the file
    try:
        with open('apicalltracker.txt', 'r') as file:
            lines = file.readlines()

        # Parse the current value from the second line
        current_value_line = lines[1].strip()
        current_value_start = current_value_line.rfind(' ') + 1
        current_value = int(current_value_line[current_value_start:])
    except (FileNotFoundError, IndexError, ValueError):
        # If the file doesn't exist or is incorrectly formatted, start with 0
        current_value = 0

    # Increment the value
    current_value += 1

    # Update the value in the file
    lines[1] = f'Number of API calls this month: {current_value}\n'

    with open('apicalltracker.txt', 'w') as file:
        file.writelines(lines)

    search_for_arbitrage(amount, detailed, ignore_draw_odds)
<u>Description</u>: the function to run the entire program. Utilises all of the functions previous of this to grab data from the API, parse it, determine the best bookmakers and if there is a arbitrage opportunity or not. 

<u>Parameters</u>:
- amount: the amount of money a user wants to test the program with. Will be used to calculate the required betting amounts and expected profits.
- detailed: whether or not we want to print the output of all matches or just the matches with arbitrage opportunities. 'True' if you want the results of every game, 'False' if you only want the results of an arbitrage (ordered by $ value).
- ignored_draw_odds: whether or not we want to ignore the odds for drawing in our calculation for an arbitrage. Highly recommended that we do NOT ignore the draw odds. 

<u>Returns</u>: 
- None

In [9]:
def search_for_arbitrage(amount: int, database_name: str, detailed: bool, show_live: bool) -> None:
    """ 
    Searches for arbitrage opportunities in the database based on various parameters.
    """    
    #Choose which dataset to pull from and increment the API tracker if necessary
    df_api, pull_from = choose_dataset(database_name)
    if database_name == "The Odds API":
        increment_tracker()
    
    #Intialise variables     
    start_time = time.monotonic()
    stake = amount
    list_arbitrage_opportunites = []
    number_of_games = len(df_api)
    df_results = pd.DataFrame(columns = ['expected_profit', 'arbitrage_data'])

    #Keep track of game number for clarity in output later 
    game_number = 0

    #Iterate through each of the games 
    for index, game in df_api.iterrows():
        game_number += 1

        #Convert the commence_time into AEDT
        formatted_aedt_time = UTC_to_AEDT(game['commence_time'])

        #Intialise some useful match details
        match_details = f"{game['home_team']} vs {game['away_team']} ({game['sport_title']})\nMatch Time: {formatted_aedt_time}"

        #Intialise empty data frame 
        df_bookmaker = pd.DataFrame(columns = ['bookmaker', 'prices', 'team1', 'team2', 'market_key', 'last_updated'])

        #Some games have no bookies (no idea why), this is to account for that.
        bookie_found = False
        
        #For each game, parse all of the bookmaker's data 
        for bookmaker in game['bookmakers']:
            parse_bookmaker_data(bookmaker, df_bookmaker)
            bookie_found = True 

        if bookie_found:
            #For each game, determine the bookmaker with the best odds
            df_best_team1_odds, df_best_team2_odds , df_best_draw_odds = find_best_odds(df_bookmaker)
            
            #Now calculate if an arbitrage opportunity exists 
            implied_probability, team1_probability, team2_probability, draw_probability = calculate_arbitrage(df_best_team1_odds, df_best_team2_odds, df_best_draw_odds)

            #Display the results 
            display_results(detailed, df_results, implied_probability, df_best_team1_odds, df_best_team2_odds, df_best_draw_odds, match_details, game_number, number_of_games, team1_probability, team2_probability, stake, list_arbitrage_opportunites, draw_probability, show_live)
        
    end_time = time.monotonic()
    print(f"---------------------------------------------------------------------\n{pull_from}\nArbitrage Opportunities: {list_arbitrage_opportunites}\nSettings Used: ['Wager Amount': ${stake}, 'Detailed' = {detailed}, 'Show live games?' = {show_live}]\nElapsed Time: {round(end_time - start_time,6)} seconds\n")

## Main ##
```
search_for_arbitrage(amount, database_name, detailed, show_live)
```
The line of code below runs the **entire program/Jupyter Notebook**. Change the parameters below to your required specificities then run the code cell below. It will then display the results of the API call.

- <u> amount</u> to the amount of dollars ($) you are willing to wager.
- <u> database_name</u> choose if we want to run the program with a text file database or drawing from a live API call.  
    - "The Odds API" --> pull live data from The Odds API.
    - "exampleAPIcall" --> pull data from the text file "exampleAPIcall.txt" (originally pulled from The Odds API).
    - "exampleAPIcall2" --> pull data from the text file "exampleAPIcall.txt" (originally pulled from The Odds API).
    - "nobookieAPIcall" --> pull data from the text file "nobookieAPIcall.txt" containing an example of a game with no bookie (originally pulled from The Odds API).  
- <u> detailed</u> to 'True' if you want the results of every game or 'False' if you only want to display arbitrage opportunites ordered by their expected payout (ordered by $ value).  
- <u> show_live</u> to 'True' if you want to show all games from the dataset or 'False' if you only want matches that are not live.  
---

**Function Presets**  
(Default) Display the results of non-live games with arbitrage opportunities ordered by their expected payout.  
<code> search_for_arbitrage(100, "The Odds API", False, False) </code>

Display the results of all games with arbitrage opportunities ordered by their expected payout regardless of commence time.   
<code> search_for_arbitrage(100, "The Odds API", False, True) </code> 

Display the results of all games regardless of commence time.  
<code> search_for_arbitrage(100, "The Odds API", True, True) </code>




In [10]:
search_for_arbitrage(100, "The Odds API", False, False)

---------------------------------------------------------------------
Data pulled from API: 'The Odds API'
Arbitrage Opportunities: []
Settings Used: ['Wager Amount': $100, 'Detailed' = False, 'Show live games?' = False]
Elapsed Time: 0.109 seconds



---